In [1]:
import os
import shutil
import pprint
import random
import numpy as np
import pandas as pd
import seaborn as sns
import cv2 as cv
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array, array_to_img
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

In [2]:
import tensorflow_addons as tfa
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
data_dir = '../input/leaf-disease-dataset-combination/image data/'
reg_dir = os.path.join(data_dir, 'validation')
train_dir = os.path.join(data_dir, 'train')

In [4]:
if not os.path.isdir('train'):
    os.mkdir('train')

In [5]:
for dir in os.listdir(reg_dir):
    if dir != 'Cassava':
        sub_dir_path = os.path.join(reg_dir, dir)

        for subclass in os.listdir(sub_dir_path):
            dst_dir = os.path.join('train',f"{dir}-{subclass}")

            if not os.path.isdir(dst_dir):
                os.mkdir(dst_dir)

            for key, file in enumerate(os.listdir(os.path.join(sub_dir_path, subclass))):
                src = os.path.join(sub_dir_path, subclass, file)
                dst = os.path.join(dst_dir, file)
                shutil.copyfile(src, dst)

                print(f'{key + 1}/{len(os.listdir(os.path.join(sub_dir_path, subclass)))}', end='\r')

            print(f'{os.path.join(reg_dir,dir, subclass)}-->{dst_dir}')

In [6]:
for dir in os.listdir(train_dir):
    if dir != 'Cassava':
        sub_dir_path = os.path.join(train_dir, dir)

        for subclass in os.listdir(sub_dir_path):
            dst_dir = os.path.join('train',f"{dir}-{subclass}")

            if not os.path.isdir(dst_dir):
                os.mkdir(dst_dir)

            for key, file in enumerate(os.listdir(os.path.join(sub_dir_path, subclass))):
                src = os.path.join(sub_dir_path, subclass, file)
                dst = os.path.join(dst_dir, file)
                shutil.copyfile(src, dst)

                print(f'{key + 1}/{len(os.listdir(os.path.join(sub_dir_path, subclass)))}', end='\r')

            print(f'{os.path.join(train_dir,dir, subclass)}-->{dst_dir}')

In [7]:
loacl_train_dir = 'train'

In [8]:
dp_ = {}
sizes = []

for top in os.listdir(loacl_train_dir):
    path = os.path.join(loacl_train_dir, top)
    dp_[top] = len(os.listdir(path))
    sizes.append(len(os.listdir(path)))

pprint.pprint(dp_)

In [9]:
for top in os.listdir(loacl_train_dir):
    path = os.path.join(loacl_train_dir, top)

    if len(os.listdir(path)) == 0:
        print(f'{path} -- removed')
        os.rmdir(path)

In [10]:
def change_contrast(image, lower, upper, copies=1):
    copies = [tf.image.random_contrast(image, lower=lower, upper=upper) for _ in range(copies)]
    return copies


def change_brightness(image, delta, copies=1):
    copies = [tf.image.random_brightness(image, max_delta=delta) for _ in range(copies)]
    return copies


def change_hue(image, delta, copies=1):
    copies = [tf.image.random_hue(image, max_delta=delta) for _ in range(copies)]
    return copies


def gamma_transformation(image, gamma=0.3, copies=1):
    low = 1 - gamma
    up = 1 + gamma
    copies = [tf.image.adjust_gamma(image, gamma=np.random.uniform(low, up, 1)) for _ in range(copies)]
    return copies

def load_img_to_array(path):
    return img_to_array(load_img(path))


def resize(image, size):
    return tf.image.resize(image, size)


def bounding_boxes(offsets, dim):
    boxes = []

    for i in offsets:
        offset_height, offset_width = i
        target_height, target_width = dim
        boxes.append([offset_height, offset_width, target_height, target_width])

    return boxes


def random_sectioning(image, offsets, dims):
    boxes = bounding_boxes(offsets, dims)
    image_sections = []

    for box in boxes:
        if random.choice([True, False]):
            section = tf.image.crop_to_bounding_box(image, box[0], box[1], box[2], box[3])
            image_sections.append(section)

    return image_sections


def aggressive_cropping(image, copies, crop_window, resize_smallest_side, output_shape):
    global img, resized_copies, crops

    if isinstance(resize_smallest_side, int):
        img = resize(image, (resize_smallest_side, resize_smallest_side))

    if isinstance(resize_smallest_side, (list, tuple)):
        resized_copies = [tf.image.resize(image, (size, size)) for size in resize_smallest_side]

    if isinstance(crop_window, int):
        if isinstance(resize_smallest_side, int):
            crops = [tf.image.random_crop(img, crop_window) for _ in range(copies)]
        elif isinstance(resize_smallest_side, (list, tuple)):
            crops = [tf.image.random_crop(img_, crop_window) for _ in range(copies) for img_ in
                     resized_copies]

    elif isinstance(crop_window, (list, tuple)):
        if isinstance(resize_smallest_side, int):
            crops = [tf.image.random_crop(img, crop_window) for _ in range(copies)]
        elif isinstance(resize_smallest_side, (list, tuple)):
            crops = [tf.image.random_crop(img_, crop_window) for _ in range(copies) for img_ in resized_copies]

    return [resize(crop_img, output_shape) for crop_img in crops]


def change_contrast(image, lower, upper, copies=1):
    copies = [tf.image.random_contrast(image, lower=lower, upper=upper) for _ in range(copies)]
    return copies


def change_brightness(image, delta, copies=1):
    copies = [tf.image.random_brightness(image, max_delta=delta) for _ in range(copies)]
    return copies


def change_hue(image, delta, copies=1):
    copies = [tf.image.random_hue(image, max_delta=delta) for _ in range(copies)]
    return copies


def gamma_transformation(image, gamma=0.3, copies=1):
    low = 1 - gamma
    up = 1 + gamma
    copies = [tf.image.adjust_gamma(image, gamma=np.random.uniform(low, up, 1)) for _ in range(copies)]
    return copies

In [11]:
dir_path = []

for top in os.listdir(loacl_train_dir):
    path = os.path.join(loacl_train_dir, top)

    if 1000 < len(os.listdir(path)) < 3000:
        dir_path.append(path)

dir_path

In [12]:
for file_dir in dir_path:
    before = len(os.listdir(file_dir))
    size_before = len(os.listdir(file_dir))
    original_files = enumerate(os.listdir(file_dir))

    for key_, file in original_files:
        img_path = os.path.join(file_dir, file)
        img = load_img(img_path)
        height, width = img_to_array(img).shape[:2]
        cropped = random_sectioning(img_to_array(img),
                                    [[0, 0], [height // 2, 0], [0, width // 2], [height // 2, width // 2],
                                     [height // 4, width // 4]],
                                    [height // 2, width // 2])

        for key, section in enumerate(cropped):
            if len(os.listdir(file_dir)) > 5555:
                break

            else:
                section_file_name = f"sec-crop-{key}-{file}"
                dst_path = os.path.join(file_dir, section_file_name)
                array_to_img(section).save(dst_path)

        print(f'{key_ + 1}/{size_before}', end='\r')

    print(f'{file_dir} -- Done -- {before}-->{len(os.listdir(file_dir))}')

In [13]:
dir_path_small = []

for top in os.listdir(loacl_train_dir):
    path = os.path.join(loacl_train_dir, top)

    if len(os.listdir(path)) < 1000:
        dir_path_small.append(path)

dir_path_small

In [14]:
for file_dir in dir_path_small:
    before = len(os.listdir(file_dir))
    size_before = len(os.listdir(file_dir))
    original_files = enumerate(os.listdir(file_dir))

    for key_, file in original_files:
        img_path = os.path.join(file_dir, file)
        img = load_img(img_path)
        cropped = aggressive_cropping(img_to_array(img), 2, (128, 128, 3), [256, 288, 320, 352], (128, 128))

        for key, section in enumerate(cropped):
            if len(os.listdir(file_dir)) > 5555:
                break

            else:
                section_file_name = f"agr-crop-{key}-{file}"
                dst_path = os.path.join(file_dir, section_file_name)
                array_to_img(section).save(dst_path)

        print(f'{key_ + 1}/{size_before}', end='\r')

    print(f'{file_dir} -- Done -- {before}-->{len(os.listdir(file_dir))}')

In [15]:
for tree in os.listdir(loacl_train_dir):
    tree_path = os.path.join(loacl_train_dir, tree)
    original_files = os.listdir(tree_path)

    for key_, file in enumerate(original_files):
        img = img_to_array(load_img(os.path.join(tree_path, file)))

        array_to_img(change_contrast(img, 0.5, 1.5)[0]).save(os.path.join(tree_path, f'con-man-{key_}-{file}'))
        array_to_img(change_brightness(img, 0.3)[0]).save(os.path.join(tree_path, f'b-man-{key_}-{file}'))
        array_to_img(change_hue(img, 0.5)[0]).save(os.path.join(tree_path, f'hue-man-{key_}-{file}'))
        array_to_img(gamma_transformation(img, 0.6)[0]).save(os.path.join(tree_path, f'gamma-man-{key_}-{file}'))

        print(f'{key_ + 1}/{len(original_files)}', end='\r')

    print(f'{tree_path} -- Done -- {len(original_files)}-->{len(os.listdir(tree_path))}')

In [16]:
generator = ImageDataGenerator(rescale=1/255.,
                              validation_split=0.3)

train_batch = generator.flow_from_directory(directory='train',
                                           target_size=(224,224),
                                           subset='training')

validation_batch = generator.flow_from_directory(directory='train',
                                           target_size=(224,224),
                                           subset='validation')

In [17]:
def inception(x, filters, projection, classes=None, aux=False, name=None, aux_name=None):
    f_1x1, f_3x3, f_3x3_reduce, f_5x5, f_5x5_reduce = filters
    x1 = Conv2D(filters=f_1x1, kernel_size=(1, 1), strides=(1, 1), activation=relu, padding='same')(x)
    x3_reducer = Conv2D(filters=f_3x3_reduce, kernel_size=(1, 1), strides=(1, 1), activation=relu, padding='same')(x)
    x5_reducer = Conv2D(filters=f_5x5_reduce, kernel_size=(1, 1), strides=(1, 1), activation=relu, padding='same')(x)
    pool = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x)

    x3 = Conv2D(filters=f_3x3, kernel_size=(3, 3), strides=(1, 1), activation=relu, padding='same')(x3_reducer)
    x5 = Conv2D(filters=f_5x5, kernel_size=(5, 5), strides=(1, 1), activation=relu, padding='same')(x5_reducer)
    proj = Conv2D(filters=projection, kernel_size=(1, 1), strides=(1, 1), activation=relu, padding='same')(pool)

    x = concatenate([x1, x3, x5, proj], axis=3, name=name)

    return x


def model_builder(shape, classes):
    input_layer = Input(shape=shape)
    x = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), activation=relu, padding='same')(input_layer)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters=64, kernel_size=(1, 1), strides=(1, 1), activation=relu, padding='same')(x)
    x = Conv2D(filters=192, kernel_size=(3, 3), strides=(1, 1), activation=relu, padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = inception(x, [64, 128, 96, 32, 16], projection=32, name='inception_3a')
    x = inception(x, [128, 192, 128, 96, 32], projection=64, name='inception_3b')
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = inception(x, [192, 208, 96, 48, 16], projection=64, name='inception_4a')

    aux_1 = AveragePooling2D(pool_size=(5, 5), strides=(3, 3), padding='valid')(x)
    aux_1 = Conv2D(filters=128, kernel_size=(1, 1), strides=(1, 1), activation=relu, padding='valid')(aux_1)
    aux_1 = Dense(units=1024, activation=relu)(aux_1)
    aux_1 = Dropout(rate=0.7)(aux_1)
    aux_1 = Flatten()(aux_1)
    aux_out1 = Dense(units=classes, activation=softmax, name='aux_out1')(aux_1)

    x = inception(x, [160, 224, 112, 64, 24], projection=64, name='inception_4b')
    x = inception(x, [128, 256, 128, 64, 24], projection=64, name='inception_4c')
    x = inception(x, [112, 288, 144, 64, 32], projection=64, name='inception_4d')
    x = inception(x, [256, 320, 160, 128, 32], projection=128, name='inception_4e')

    aux_2 = AveragePooling2D(pool_size=(5, 5), strides=(3, 3), padding='valid')(x)
    aux_2 = Conv2D(filters=128, kernel_size=(1, 1), strides=(1, 1), activation=relu, padding='valid')(aux_2)
    aux_2 = Dense(units=1024, activation=relu)(aux_2)
    aux_2 = Dropout(rate=0.7)(aux_2)
    aux_2 = Flatten()(aux_2)
    aux_out2 = Dense(units=classes, activation=softmax, name='aux_out2')(aux_2)

    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = inception(x, [256, 320, 160, 128, 32], projection=128, name='inception_5a')
    x = inception(x, [384, 384, 192, 128, 48], projection=128, name='inception_5b')
    x = AveragePooling2D(pool_size=(7, 7), strides=(1, 1))(x)
    x = Dropout(rate=0.4)(x)
    x = Flatten()(x)
    output_layer = Dense(units=classes, activation=softmax, name='main_out')(x)

    model = Model(input_layer, [output_layer, aux_out1, aux_out2])
    model.compile(optimizer=Adam(), loss=categorical_crossentropy,
                  loss_weights={'main_out': 1, 'aux_out1': 0.3, 'aux_out2': 0.3},
                  metrics=['accuracy', tfa.metrics.F1Score(num_classes=classes, threshold=0.5)])
    model.summary()

    return model

In [ ]:
inception = model_builder(shape=(224, 224, 3), classes=39)

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=1e-1, patience=15)
early_stop = EarlyStopping(monitor='val_loss', patience=8)

In [ ]:
inception_history = inception.fit(x=train_batch,
                              epochs=100,
                              steps_per_epoch=6000,
                              validation_steps=2000,
                              validation_data=validation_batch,
                              callbacks=[reduce_lr, early_stop])

In [ ]:
temp = pd.DataFrame(inception_history.history)
temp.to_pikkel('inception_history.pkl')

In [ ]:
inception.save('inception.hdf5')